# Model Selection

https://medium.com/@ebimsv/ml-series-day-42-statistical-tests-for-model-comparison-4f5cf63da74a

In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Output
from IPython.display import display
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy import stats
from math import log

import warnings
warnings.filterwarnings('ignore')

# Modelos de clasificación
from sklearn.linear_model import (
    LogisticRegression, RidgeClassifier, SGDClassifier, Perceptron, PassiveAggressiveClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, 
    ExtraTreesClassifier, BaggingClassifier, HistGradientBoostingClassifier
)
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

# Cargar los conjuntos de datos desde archivos CSV
X = pd.read_csv("datasets/X_train.csv")
y = pd.read_csv("datasets/y_train.csv").values.ravel()


# Lista de modelos disponibles
model_options = {
    "Logistic Regression": LogisticRegression(),
    "Ridge Classifier": RidgeClassifier(),
    "SGD Classifier": SGDClassifier(),
    "Perceptron": Perceptron(),
    "Passive Aggressive": PassiveAggressiveClassifier(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM (SVC)": SVC(),
    "SVM (NuSVC)": NuSVC(),
    "SVM (LinearSVC)": LinearSVC(),
    "Random Forest": RandomForestClassifier(),
    #"Gradient Boosting": GradientBoostingClassifier(),
    "Hist Gradient Boosting": HistGradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
    "Bagging": BaggingClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "Bernoulli Naive Bayes": BernoulliNB(),
    "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),
    "Quadratic Discriminant Analysis": QuadraticDiscriminantAnalysis(),
    "MLP (Neural Network)": MLPClassifier()
}

# Crear widgets para selección de modelos
model_1_selector = widgets.Dropdown(options=list(model_options.keys()), description="Model 1")
model_2_selector = widgets.Dropdown(options=list(model_options.keys()), description="Model 2")
run_button = widgets.Button(description="Run Evaluation")
output = widgets.Output()

def get_pipeline(model):
    return make_pipeline(StandardScaler(), model)

def evaluate_models(b):
    with output:
        output.clear_output()
        model_1 = model_options[model_1_selector.value]
        model_2 = model_options[model_2_selector.value]
        
        # Calcular precisión con validación cruzada
        acc_model_1 = cross_val_score(get_pipeline(model_1), X, y, cv=5, scoring='accuracy')
        acc_model_2 = cross_val_score(get_pipeline(model_2), X, y, cv=5, scoring='accuracy')
        print(f"Cross-Validated Accuracy Model 1 ({model_1_selector.value}): {acc_model_1.mean()}")
        print(f"Cross-Validated Accuracy Model 2 ({model_2_selector.value}): {acc_model_2.mean()}")
        
        # Prueba estadística (t-test)
        t_stat, p_value = stats.ttest_rel(acc_model_1, acc_model_2)
        print(f"T-Statistic: {t_stat}")
        print(f"P-Value: {p_value}")
        if p_value < 0.05:
            print("We reject the null hypothesis: The difference in performance is statistically significant.")
        else:
            print("We fail to reject the null hypothesis: The difference in performance is not statistically significant.")
        
        # Calcular AIC y BIC
        log_likelihood_model_1 = -log(acc_model_1.mean())
        log_likelihood_model_2 = -log(acc_model_2.mean())
        n = len(y)
        k_model_1 = 1  # Se usa un valor por defecto para simplificar
        k_model_2 = 1
        
        aic_1 = 2*k_model_1 - 2*log_likelihood_model_1
        bic_1 = np.log(n)*k_model_1 - 2*log_likelihood_model_1
        aic_2 = 2*k_model_2 - 2*log_likelihood_model_2
        bic_2 = np.log(n)*k_model_2 - 2*log_likelihood_model_2
        
        print(f"AIC Model 1 ({model_1_selector.value}): {aic_1}")
        print(f"BIC Model 1 ({model_1_selector.value}): {bic_1}")
        print(f"AIC Model 2 ({model_2_selector.value}): {aic_2}")
        print(f"BIC Model 2 ({model_2_selector.value}): {bic_2}")

run_button.on_click(evaluate_models)

# Mostrar widgets
display(model_1_selector, model_2_selector, run_button, output)


Dropdown(description='Model 1', options=('Logistic Regression', 'Ridge Classifier', 'SGD Classifier', 'Percept…

Dropdown(description='Model 2', options=('Logistic Regression', 'Ridge Classifier', 'SGD Classifier', 'Percept…

Button(description='Run Evaluation', style=ButtonStyle())

Output()